In [6]:
import xarray
import numpy as np
import zarr
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster, progress
from rechunker import rechunk
from tqdm import tqdm
import gc

In [2]:
hres = xarray.open_zarr(
    "/scratch/snx3000/acarpent/EumetsatData/SEVIRI_WGS_2016-2022_RSS.zarr",
    chunks={},
)
# engine='zarr',)
# chunks='auto')
hres = hres.drop_duplicates(dim="time")

hres.drop('spatial_ref')

<xarray.Dataset>
Dimensions:  (time: 222098, y: 658, x: 736)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 ... 2022-12-31T23:45:00
  * x        (x) float64 -7.775 -7.725 -7.675 -7.625 ... 28.83 28.88 28.93 28.98
  * y        (y) float64 28.98 29.03 29.08 29.12 ... 61.68 61.73 61.78 61.83
Data variables:
    IR_016   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_039   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_087   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_097   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_108   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_120   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_134   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    VIS006   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    VIS008   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    WV_062   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    WV_073   (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>

In [ ]:
a = output_store.isel(time=-2).__xarray_dataarray_variable__
import matplotlib.pyplot as plt
for i in range(12):
    plt.figure()
    plt.title(output_store.data[i].values)
    plt.imshow(a[i,:,:])
    plt.colorbar()

In [7]:
slice_lengths = int(len(hres.time) / 500)
already_rechunked = 0
compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)

for i in tqdm(range(0, len(hres.time), slice_lengths)):
    
    if i + slice_lengths <= already_rechunked:
        print(f'skipped {i}')
        continue
        
    if i < already_rechunked and i + slice_lengths > already_rechunked:
        hres_slice = hres_isel(time=slice(already_rechunked, i+slice_lengths))
    else:
        hres_slice = hres.isel(time=slice(i, i + slice_lengths))
        
    for var in hres_slice:
        hres_slice[var].encoding = {'dtype':'<f2',
                                   'compressor': compressor}
    
    hres_slice = hres_slice.drop('spatial_ref')
    hres_slice = hres_slice.to_dataarray(dim="channel", name='channel_data').astype(np.float16)
    target_store = "/scratch/snx3000/kschuurm/ZARR/SEVIRI_new.zarr"
    target_chunks = {"channel": -1, "time": 1, "x": -1, "y": -1}
    hres_slice = hres_slice.chunk(target_chunks)
    if i == 0:
        hres_slice.to_zarr(target_store, mode="w")
    else:
        hres_slice.to_zarr(target_store, append_dim="time")
    del hres_slice
    gc.collect()

100%|██████████| 501/501 [50:48<00:00,  6.08s/it]


In [9]:
output_store = xarray.open_zarr("/scratch/snx3000/kschuurm/ZARR/SEVIRI_new.zarr")
output_store

<xarray.Dataset>
Dimensions:       (channel: 11, time: 222098, y: 658, x: 736)
Coordinates:
  * channel       (channel) object 'IR_016' 'IR_039' ... 'WV_062' 'WV_073'
  * time          (time) datetime64[ns] 2016-01-01 ... 2022-12-31T23:45:00
  * x             (x) float64 -7.775 -7.725 -7.675 -7.625 ... 28.88 28.93 28.98
  * y             (y) float64 28.98 29.03 29.08 29.12 ... 61.73 61.78 61.83
Data variables:
    channel_data  (channel, time, y, x) float16 dask.array<chunksize=(11, 1, 658, 736), meta=np.ndarray>